Components are the building blocks of KFP pipelines. A component is a remote function definition; it specifies inputs, has user-defined logic in its body, and can create outputs. When the component template is instantiated with input parameters, we call it a task.

KFP provides two high-level ways to author components: **Python Components** and **Container Components**.

Python Components are a convenient way to author components implemented in pure Python. There are two specific types of Python components: **Lightweight Python Components** and **Containerized Python Components**.

Container Components expose a more flexible, advanced authoring approach by allowing you to define a component using an arbitrary container definition. This is the recommended approach for components that are not implemented in pure Python.

**Importer Components** are a special “pre-baked” component provided by KFP which allows you to import an artifact into your pipeline when that artifact was not created by tasks within the pipeline.

## Lightweight Python Components
Lightweight Python Components are constructed by decorating Python functions with the `@dsl.component` decorator. The `@dsl.component` decorator transforms your function into a KFP component that can be executed as a remote function.

To decorate a function with the `@dsl.component` decorator it must meet two requirements:
1. **Type annotations**: The function inputs and outputs must have valid KFP type annotations.  
  There are two categories of inputs and outputs in KFP: parameters and artifacts. There are specific types of parameters and artifacts within each category. Every input and output will have a specific type indicated by its type annotation.  
  Valid parameter annotations include Python’s built-in `int`, `float`, `str`, `bool`, `typing.Dict`, and `typing.List`.
2. **Hermetic**: The Python function may not reference any symbols defined outside of its body. All constants or symbols and imports must be included in the function body.

For many realistic components, hermeticism can be a fairly constraining requirement. Containerized Python Components is a more flexible authoring approach that drops this requirement.

### dsl.component decorator arguments
#### packages_to_install 
You can pass a list of requirements to `packages_to_install` and the component will install these packages at runtime before executing the component function.  
**Note**: As a production software best practice, prefer using Containerized Python Components when your component specifies `packages_to_install` to eliminate installation of your dependencies at runtime.

#### pip_index_urls
`pip_index_urls` exposes the ability to pip install `packages_to_install` from package indices other than the default PyPI.org.  
When you set `pip_index_urls`, KFP passes these indices to `pip install`’s `--index-url` and `--extra-index-url` options. It also sets each index as a `--trusted-host`.  
Note that when you set `pip_index_urls`, KFP does not include `'https://pypi.org/simple'` automatically. If you wish to pip install packages from a private repository and the default public repository, you should include both the private and default URLs.
```python
@dsl.component(packages_to_install=['custom-ml-package==0.0.1', 'numpy==1.21.6'],
               pip_index_urls=['http://myprivaterepo.com/simple', 'http://pypi.org/simple'],
)
```

#### base_image
When you create a Lightweight Python Component, your Python function code is extracted by the KFP SDK to be executed inside a container at pipeline runtime. By default, the container image used is `python:3.7`. You can override this image by providing an argument to `base_image`. This can be useful if your code requires a specific Python version or other dependencies not included in the default image.

#### install_kfp_package
`install_kfp_package` can be used together with `pip_index_urls` to provide granular control over installation of the kfp package at component runtime.  
By default, Python Components install `kfp` at runtime. If `install_kfp_package` is `False`, `kfp` will not be installed via the normal automatic mechanism. Instead, you can use `packages_to_install` and `pip_index_urls` to install a different version of `kfp`, possibly from a non-default pip index URL.  
Note that setting `install_kfp_package` to `False` is rarely necessary and is discouraged for the majority of use cases.

## Compose components into pipelines
While components have three authoring approaches, pipelines have one authoring approach: they are defined with a pipeline function decorated with the `@dsl.pipeline` decorator.   
Although a KFP pipeline decoratored with the `@dsl.pipeline` decorator looks like a normal Python function, it is actually an expression of pipeline topology and control flow semantics, constructed using the KFP domain-specific language (DSL).

A pipeline definition has four parts:
1. The pipeline decorator
2. Inputs and outputs declared in the function signature
3. Data passing and task dependencies
4. Task configurations
5. Pipeline control flow

### The pipeline decorator 
KFP pipelines are defined inside functions decorated with the `@dsl.pipeline` decorator. The decorator takes three optional arguments:
- `name` is the name of your pipeline. If not provided, the name defaults to a sanitized version of the pipeline function name.
- `description` is a description of the pipeline.
- `pipeline_root` is the root path of the remote storage destination within which the tasks in your pipeline will create outputs. `pipeline_root` may also be set or overridden by pipeline submission clients.
- `display_name` is a human-readable for your pipeline.

### Pipeline inputs and outputs
Pipeline inputs and outputs are defined by the parameters and annotations in the pipeline function signature.  
Pipeline inputs are declaried via function input parameters/annotations and pipeline outputs are declared via function output annotations. Pipeline outputs will *never be declared via pipeline function input parameters*, unlike for components that use output artifacts or Container Components that use dsl.OutputPath.

### Data passing and task dependencies
When you call a component in a pipeline definition, it constructs a `PipelineTask` instance. You can pass data between tasks using the `PipelineTask’s` `.output` and `.outputs` attributes. For a task with a single unnamed output indicated by a single return annotation, access the output using `PipelineTask.output`. For tasks with multiple outputs or named outputs, access the output using `PipelineTask.outputs['<output-key>']`. In the absence of data exchange, tasks will run in parallel for efficient pipeline executions. When tasks exchange data, an execution ordering is established between those tasks. This is to ensure that upstream tasks create their outputs before downstream tasks attempt to consume those outputs.  
In some cases, you may wish to establish execution ordering in the absence of data exchange. In these cases, you can call one task’s `.after()` method on another task.

```python
@dsl.pipeline
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    b_sq_task = square(x=b)
    b_sq_task.after(a_sq_task)
```

### Special input types
There are a few special input values that you can pass to a component within your pipeline definition to give the component access to some metadata about itself. These values can be passed to input parameters typed `str`.

```python
@dsl.pipeline
def my_pipeline():
    print_op(text=dsl.PIPELINE_JOB_NAME_PLACEHOLDER)
```

There several special values that may be used in this style, including:
- `dsl.PIPELINE_JOB_NAME_PLACEHOLDER`
- `dsl.PIPELINE_JOB_RESOURCE_NAME_PLACEHOLDER`
- `dsl.PIPELINE_JOB_ID_PLACEHOLDER`
- `dsl.PIPELINE_TASK_NAME_PLACEHOLDER`
- `dsl.PIPELINE_TASK_ID_PLACEHOLDER`
- `dsl.PIPELINE_JOB_CREATE_TIME_UTC_PLACEHOLDER` (unsupported)
- `dsl.PIPELINE_JOB_SCHEDULE_TIME_UTC_PLACEHOLDER` (unsupported)
- `dsl.PIPELINE_ROOT_PLACEHOLDER` (unsupported)

### Task configurations
The KFP SDK exposes several platform-agnostic task-level configurations via task methods. All platform-agnostic task-level configurations are set using `PipelineTask` methods.

```python
@dsl.component
def print_env_var():
    import os
    print(os.environ.get('MY_ENV_VAR'))

@dsl.pipeline()
def my_pipeline():
    task = print_env_var()
    task.set_env_variable('MY_ENV_VAR', 'hello')
```

Task-level configuration methods can also be chained:
```python
print_env_var().set_env_variable('MY_ENV_VAR', 'hello').set_env_variable('OTHER_VAR', 'world')
```

The KFP SDK provides the following task methods for setting task-level configurations:
- `.add_accelerator_type`
- `.set_accelerator_limit`
- `.set_cpu_limit`
- `.set_memory_limit`
- `.set_env_variable`
- `.set_caching_options`
- `.set_display_name`
- `.set_retry`
- `.ignore_upstream_failure` (unsupported)

### Pipelines as components
Pipelines can themselves be used as components in other pipelines, just as you would use any other single-step component in a pipeline.

## Containerized Python Components
Containerized Python Components extend Lightweight Python Components by relaxing the constraint that Lightweight Python Components be hermetic (i.e., fully self-contained). This means Containerized Python Component functions can depend on symbols defined outside of the function, imports outside of the function, code in adjacent Python modules, etc. To achieve this, the KFP SDK provides a convenient way to package your Python code into a container.

As a production software best practice, component authors should prefer Containerized Python Components to Lightweight Python Components when their component specifies `packages_to_install`, since the KFP SDK will install these dependencies into the component’s image when it is built, rather than at task runtime.

Following are the steps to use Containerized Python Components by modifying Lightweight Python Components:
1. **Source code setup**: Start by creating an empty `src/` directory to contain your source code. Then add `src/my_component.py` with component definition. If there are any functions in other files, you can import and use them following regular import and use methods in the function body.

2. **Modify the dsl.component decorator**: provide `base_image` and `target_image` arguments to the `@dsl.component` decorator of your component in `src/my_component.py`.  
  Setting `target_image` both (a) specifies the tag for the image built, and (b) instructs KFP to run the decorated Python function in a container that uses the image with that tag.  
  In a Containerized Python Component, `base_image` specifies the base image that KFP will use when building new container image. Specifically, KFP uses the `base_image` argument for the FROM instruction in the Dockerfile used to build image. `base_image` will default to `'python:3.11'` if omitted.   
  ```python
  @dsl.component(base_image='python:3.11',
               target_image='gcr.io/my-project/my-component:v1')
  ```

3. **Build the component**: Now that the code is in a standalone directory and you’ve specified a target image, you can conveniently build an image using the `kfp component build` CLI command:  
  If you have a configured Docker to use a private image registry, you can replace the --no-push-image flag with --push-image to automatically push the image after building.
  ```bash
  kfp component build src/ --component-filepattern my_component.py --no-push-image
  ```

4. **Use the component in a pipeline**: Use the component in a pipeline by importing outside the pipeline definition as following:
  ```python
  from src.my_component import component_name
  ```

## Container components
In KFP, each task execution corresponds to a container execution. This means that all components are defined by an `image`, `command`, and `args`.

Python Components are unique because they abstract most aspects of the container definition away from the user, making it convenient to construct components that use pure Python. Under the hood, the KFP SDK sets the `image`, `commands`, and `args` to the values needed to execute the Python component for the user.

**Container Components, unlike Python Components, enable component authors to set the `image`, `command`, and `args` directly**. This makes it possible to author components that execute shell scripts, use other languages and binaries, etc., all from within the KFP Python SDK.

### A simple Container Component
To create a Container Components, use the `dsl.container_component` decorator and create a function that returns a `dsl.ContainerSpec` object. `dsl.ContainerSpec` accepts three arguments: `image`, `command`, and `args`. 

```python
@dsl.container_component
def say_hello():
    return dsl.ContainerSpec(image='alpine', command=['echo'], args=['Hello'])
```

### Use component inputs
The parameters and annotations in the Container Component function declare the component’s interface.
- Use `command` to specify a command and `args` to provide *fstring* with parameters. On compiling the component, parameters are replaced with placeholders. At runtime, these placeholders are replaced with the actual values for parameters provided to the component.
- Another way to implement this component is to use `sh -c` to read the commands from a single string and pass the name as an argument. This approach tends to be more flexible, as it readily allows chaining multiple commands together.

### Create component outputs 
Unlike Python functions, containers do not have a standard mechanism for returning values. To enable Container Components to have outputs, KFP requires you to write outputs to a file inside the container. KFP will read this file and persist the output.

To return an output string from a component, you can add an output parameter to the function using a `dsl.OutputPath(str)` annotation. At runtime, parameters annotated with `dsl.OutputPath` will be provided a system-generated path as an argument. Your component logic should write the output value to this path as JSON. You can fill in the `command` and `args` to write the output.

```python
def say_hello(name: str, greeting: dsl.OutputPath(str)):
    return dsl.ContainerSpec(
        image='alpine',
        command=[
            'sh', '-c', '''RESPONSE="Hello, $0!"\
                            && echo $RESPONSE\
                            && mkdir -p $(dirname $1)\
                            && echo $RESPONSE > $1
                            '''
        ],
        args=[name, greeting])
```

### Use in a pipeline
Using container component in a pipeline is very similar to python component with one key difference: since the container component uses named output parameter, we access it and return it from a pipeline using `task_name.outputs['parameter_name']`, instead of `task_name.output`.

Note that you will never provide output parameters to components when constructing your pipeline; output parameters are always provided automatically by the backend at runtime.

### Special placeholders
Each of the three component authoring styles automatically handle data passing into your component via placeholders in the container `command` and `args`. Container Components also enable you to directly utilize two special placeholders if you wish: `dsl.ConcatPlaceholder` and `dsl.IfPresentPlaceholder`.

You may only use these placeholders in the `dsl.ContainerSpec` returned from a Container Component authored via the `@dsl.container_component decorator`.

#### dsl.ConcatPlaceholder
When you provide a container `command` or container `args` as a list of strings, each element in the list is concatenated using a space separator, then issued to the container at runtime. Concatenating one input to another string without a space separator requires special handling provided by the `dsl.ConcatPlaceholder`.

`dsl.ConcatPlaceholder` takes one argument, `items`, which may be a list of any combination of static strings, upstream outputs, pipeline parameters, or other instances of `dsl.ConcatPlaceholder` or `dsl.IfPresentPlaceholder`. At runtime, these strings will be concatenated together without a separator.

#### dsl.IfPresentPlaceholder
`dsl.IfPresentPlaceholder` is used to conditionally provide command line arguments. The `dsl.IfPresentPlaceholder` takes three arguments: `input_name`, `then`, and optionally `else_`. 

If the component is passed with a value for the optonal parameter mentioned in `input_name`, the component will pass arguments in `then` to the executable command/script. If the optional parameter is not provided, arguments are omitted.

The third parameter `else_` can be used to provide a default value to fall back to if `input_name` is not provided. 

Arguments to `then` and `else_` may be a list of any combination of static strings, upstream outputs, pipeline parameters, or other instances of `dsl.ConcatPlaceholder` or `dsl.IfPresentPlaceholder`.

## Special Case: Importer Components
**Import artifacts from outside your pipeline**  

Unlike the other three authoring approaches, an importer component is not a general authoring style but a pre-baked component for a specific use case: loading a machine learning artifact from from a URI into the current pipeline and, as a result, into ML Metadata.

If you wish to use an existing artifact that was not generated by a task in the current pipeline, you can use a `dsl.importer` component to load the artifact from its URI.

You do not need to write an importer component; it can be imported from the `dsl` module and used directly.

```python
importer_task = dsl.importer(
        artifact_uri='gs://ml-pipeline-playground/shakespeare1.txt',
        artifact_class=dsl.Dataset,
        reimport=True,
        metadata={'date': task.output})
```

In addition to an `artifact_uri` argument, you must provide an `artifact_class` argument to specify the type of the artifact.

The `importer` component permits setting artifact metadata via the `metadata` argument. Metadata can be constructed with outputs from upstream tasks.

You may also specify a boolean `reimport` argument. If `reimport` is `False`, KFP will check to see if the artifact has already been imported to ML Metadata and, if so, use it. This is useful for avoiding duplicative artifact entries in ML Metadata when multiple pipeline runs import the same artifact. If `reimport` is `True`, KFP will reimport the artifact as a new artifact in ML Metadata regardless of whether it was previously imported.

## Component docstring format 
KFP allows you to document your components and pipelines using Python docstrings. For *components*, KFP can extract your component **input descriptions** and **output descriptions**. For *pipelines*, KFP can extract your pipeline **input descriptions** and **output descriptions**, as well as **a description of full pipeline**.

For the KFP SDK to correctly parse docstrings, you should write your docstrings in the KFP docstring style. The KFP docstring style is a particular variant on the Google docstring style, with the following changes:
- The `Returns: `section takes the same structure as the `Args:` section, where each return value in the `Returns:` section should take the form `<name>: <description>.` This is distinct from the typical Google docstring `Returns:` section which takes the form `<type>: <description>`, with no names for return values.
- Component outputs should be included in the `Returns:` section, even though they are declared via component function input parameters. This applies to function parameters annotated with `dsl.OutputPath` and the `Output[<Artifact>]` type marker for declaring output artifacts.
- Suggested: Type information, including which inputs are optional/required, should be omitted from the input/output descriptions. This information is duplicative of the annotations.

Note that if you provide a `description` argument to the `@dsl.pipeline` decorator, KFP will use this description instead of the docstring description.

## Load and Share Components
IR YAML serves as a portable, sharable computational template. This allows you compile and share your components with others, as well as leverage an ecosystem of existing components. “components”, here, refers to both single-step components and pipelines, which can also be used as components.

- To use an existing component, you can load it using the `components` module and use it with other components in a pipeline:

```python
loaded_comp = kfp.components.load_component_from_file('component.yaml')
```

- You can also load a component directly from a URL, such as a GitHub URL:

```python
loaded_comp = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/2.0.0/sdk/python/test_data/components/add_numbers.yaml')
```

- You can load a component from a string using `components.load_component_from_text`:

```python
with open('component.yaml') as f:
    component_str = f.read()

loaded_comp = components.load_component_from_text(component_str)
```

- As components and pipelines are persisted in the same format (IR YAML), you can also load a pipeline from a local file, URL, or string, just like you load components. Once loaded, a pipeline can be used in another pipeline.

Some libraries, such as Google Cloud Pipeline Components package and provide reusable components in a pip-installable Python package.